In [1]:
import pandas as pd

filename = "loan.csv"

loan_data = pd.read_csv(filename)

C:\Users\Bineesh\AppData\Local\Temp\ipykernel_19812\1607258213.py:5: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_data = pd.read_csv(filename)


In [2]:
loan_data.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object', length=111)

* Load data contains 111 columns

In [3]:
filename1 = "Data_Dictionary.xlsx"
data_dictionary = pd.read_excel(filename1)




In [4]:
data_dictionary.columns
loanstat = 'loanstat'
description = 'description'
data_dictionary.columns = [loanstat, description]
data_dictionary

,loanstat,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
112,verification_status,"Indicates if income was verified by LC, not ve..."
113,verified_status_joint,Indicates if the co-borrowers' joint income wa...
114,zip_code,The first 3 numbers of the zip code provided b...
115,NaN,NaN


* data dictionary contains 117 rows which corresponds to 111 columns in the load dataframe
* So need to correct this data quality issue

In [5]:
column_df = pd.DataFrame({loanstat: loan_data.columns.sort_values()})
column_df

,loanstat
0,acc_now_delinq
1,acc_open_past_24mths
2,addr_state
3,all_util
4,annual_inc
...,...
106,total_rev_hi_lim
107,url
108,verification_status
109,verification_status_joint


In [6]:
def get_column_mismatch(loan_data, data_dictionary):
    diff1 = set(loan_data.columns) - set(data_dictionary[loanstat])
    print("These are the extra columns in loan_data\n", diff1)
    diff2 = set(data_dictionary[loanstat]) - set(loan_data.columns)
    print("These are the extra info in data dictionary\n", diff2)
    #return diff1, diff2
    
get_column_mismatch(loan_data, data_dictionary)


These are the extra columns in loan_data
 {'verification_status_joint', 'total_rev_hi_lim'}
These are the extra info in data dictionary
 {nan, 'fico_range_low', 'total_rev_hi_lim \xa0', 'fico_range_high', 'last_fico_range_low', 'last_fico_range_high', 'verified_status_joint'}


* There are two columns extra in the loan data.

* There are two columns extra in the loan data.
* There is an issue with total_rev_hi_lim, ie some special characters in it
* verification_status_joint in loan_data is verified_status_joint in data dictionary
* There are nan values in the data dictionary


In [7]:
mismatched_values = {'total_rev_hi_lim \xa0': 'total_rev_hi_lim', 'verified_status_joint': 'verification_status_joint'}
data_dictionary[loanstat] = data_dictionary[loanstat].replace(mismatched_values)

get_column_mismatch(loan_data, data_dictionary)

These are the extra columns in loan_data
 set()
These are the extra info in data dictionary
 {nan, 'fico_range_low', 'fico_range_high', 'last_fico_range_low', 'last_fico_range_high'}


* 'fico_range_high',
  'fico_range_low',
  'last_fico_range_high',
  'last_fico_range_low' These 4 variables to be interpreted later. These are present in data dictionary but not present in loan data. Need to check whether these variables can be derived from existing variables

In [8]:
cleaned_data_dictionary = column_df.merge(data_dictionary, on=loanstat, how='inner')
cleaned_data_dictionary.to_csv("cleaned_data_dictionary.csv", index=False)

In [9]:
cleaned_data_dictionary.set_index(loanstat)[description].to_dict()


{'acc_now_delinq': 'The number of accounts on which the borrower is now delinquent.',
 'acc_open_past_24mths': 'Number of trades opened in past 24 months.',
 'addr_state': 'The state provided by the borrower in the loan application',
 'all_util': 'Balance to credit limit on all trades',
 'annual_inc': 'The self-reported annual income provided by the borrower during registration.',
 'annual_inc_joint': 'The combined self-reported annual income provided by the co-borrowers during registration',
 'application_type': 'Indicates whether the loan is an individual application or a joint application with two co-borrowers',
 'avg_cur_bal': 'Average current balance of all accounts',
 'bc_open_to_buy': 'Total open to buy on revolving bankcards.',
 'bc_util': 'Ratio of total current balance to high credit/credit limit for all bankcard accounts.',
 'chargeoff_within_12_mths': 'Number of charge-offs within 12 months',
 'collection_recovery_fee': 'post charge off collection fee',
 'collections_12_mth

In [27]:
loan_data.shape

(39717, 111)

In [30]:
(loan_data.isnull().sum() == loan_data.shape[0]).sum()

54

* 54 columns are completely empty


In [53]:
# get rid of 54 empty columns

columns_non_null = (loan_data.isnull().sum() != loan_data.shape[0])
columns_non_null

loan_data = loan_data.loc[:,columns_non_null]

In [38]:
loan_data.shape

(39717, 57)

In [70]:
for i in range(0, len(data_dictionary)):
    variable = data_dictionary.iloc[i,0]
    if variable in loan_data.columns:
        shape = loan_data[variable].value_counts().shape
        print(variable, shape)
#         if shape[0] == 1:
#             print(variable, shape, loan_data[variable].value_counts())

addr_state (50,)
annual_inc (5318,)
chargeoff_within_12_mths (1,)
collection_recovery_fee (2616,)
collections_12_mths_ex_med (1,)
delinq_2yrs (11,)
desc (26527,)
dti (2868,)
earliest_cr_line (526,)
emp_length (11,)
emp_title (28820,)
funded_amnt (1041,)
funded_amnt_inv (8205,)
grade (7,)
home_ownership (5,)
id (39717,)
inq_last_6mths (9,)
installment (15383,)
int_rate (371,)
issue_d (55,)
last_credit_pull_d (106,)
last_pymnt_amnt (34930,)
last_pymnt_d (101,)
loan_amnt (885,)
loan_status (3,)
member_id (39717,)
mths_since_last_delinq (95,)
mths_since_last_record (111,)
next_pymnt_d (2,)
open_acc (40,)
out_prncp (1137,)
out_prncp_inv (1138,)
pub_rec (5,)
pub_rec_bankruptcies (3,)
purpose (14,)
recoveries (4040,)
revol_bal (21711,)
revol_util (1089,)
sub_grade (35,)
tax_liens (1,)
term (2,)
title (19615,)
total_acc (82,)
total_pymnt (37850,)
total_pymnt_inv (37518,)
total_rec_int (35148,)
total_rec_late_fee (1356,)
total_rec_prncp (7976,)
url (39717,)
verification_status (3,)
zip_code (82

In [56]:
columns_with_single_unique_value = loan_data.columns[loan_data.nunique() == 1]
columns_with_single_unique_value

Index(['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq',
       'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens'],
      dtype='object')

In [57]:
loan_data[columns_with_single_unique_value].isnull().sum()

pymnt_plan                     0
initial_list_status            0
collections_12_mths_ex_med    56
policy_code                    0
application_type               0
acc_now_delinq                 0
chargeoff_within_12_mths      56
delinq_amnt                    0
tax_liens                     39
dtype: int64

In [60]:
a = loan_data[columns_with_single_unique_value].isnull().sum() == 0
a

pymnt_plan                     True
initial_list_status            True
collections_12_mths_ex_med    False
policy_code                    True
application_type               True
acc_now_delinq                 True
chargeoff_within_12_mths      False
delinq_amnt                    True
tax_liens                     False
dtype: bool

In [68]:
loan_data = loan_data.drop(columns=a[a == True].index)

In [69]:
loan_data.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'url', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'chargeoff_within_12_mths', 'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')

In [73]:
loan_data['loan_status'].value_counts()

Fully Paid     32950
Charged Off     5627
Current         1140
Name: loan_status, dtype: int64

In [74]:
column_df = pd.DataFrame({loanstat: loan_data.columns.sort_values()})
cleaned_data_dictionary = column_df.merge(data_dictionary, on=loanstat, how='inner')
cleaned_data_dictionary.set_index(loanstat)[description].to_dict()

{'addr_state': 'The state provided by the borrower in the loan application',
 'annual_inc': 'The self-reported annual income provided by the borrower during registration.',
 'chargeoff_within_12_mths': 'Number of charge-offs within 12 months',
 'collection_recovery_fee': 'post charge off collection fee',
 'collections_12_mths_ex_med': 'Number of collections in 12 months excluding medical collections',
 'delinq_2yrs': "The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years",
 'desc': 'Loan description provided by the borrower',
 'dti': 'A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.',
 'earliest_cr_line': "The month the borrower's earliest reported credit line was opened",
 'emp_length': 'Employment length in years. Possible values are between 0 and 10 where 0 means less than one ye